# Phishing Detection

## Required Packages

In [1]:
#!pip install python-whois
#!pip install googlesearch-python

In [2]:
import requests
import whois
from datetime import datetime
from googlesearch import search

In [3]:
whiteList = open("white-list.txt", "r")
blackList = open("black-list.txt", "r")

## White-List

In [4]:
whiteList = whiteList.readlines()
def checkWhiteList(url, whiteList = whiteList):
    if url in whiteList:
        return True
    return False

## Black-List

In [5]:
blackList = blackList.readlines()

In [6]:
# source for appspot = https://www.sslcertificates.top/certificate/12296
domains = ["co.in", "co.uk", "co.za", "ac.in", "ac.th", "com.br", "ac.id", "com.ec", "org.au", "org.za", "com.hk", "org.ar", "com.au", ".io", "web.app", "appspot.com", "an.r.appspot.com", "app.google", "as.r.appspot.com", "de.r.appspot.com", "df.r.appspot.com", "dt.r.appspot.com", "du.r.appspot.com", "el.r.appspot.com", "et.r.appspot.com", "ew.r.appspot.com", "ey.r.appspot.com", "ez.r.appspot.com", "lz.r.appspot.com", "nn.r.appspot.com", "nw.r.appspot.com", "nz.r.appspot.com", "oa.r.appspot.com", "rj.r.appspot.com", "thinkwithgoogle.com", "ts.r.appspot.com", "tz.r.appspot.com", "uc.r.appspot.com", "ue.r.appspot.com", "uk.r.appspot.com", "uw.r.appspot.com", "withgoogle.com", "blogspot.com", "withyoutube.com", "wl.r.appspot.com", "wm.r.appspot.com", "wn.r.appspot.com", "app.google", "appspot.com", "thinkwithgoogle.com", "withgoogle.com", "withyoutube.com"]
BlackList = []
for url in blackList:
    index = 0
    # some has http or https or nothing
    if "http" in url:
        index = url.find('://')
        url = url[3:]
    url = url[index:]
    index = url.find('/')
    if index != -1:
        url = url[:index]
    prevUrl = url
    index = url.find('.')
    while index != -1:
        prevUrl = url
        url = url[index + 1:]
        if url in domains:
            break
        index = url.find('.')
    BlackList.append(prevUrl)
blackList = BlackList
BlackList = []
def checkBlackList(url, blackList = blackList):
    if url in blackList:
        return True
    return False

## Whois

In [8]:
def check_whois(url):
    try:  
        w = whois.whois(url)
    except:
        return True
    try:
        d = w.expiration_date.year - w.creation_date.year
    except:
        try:
            d = w.expiration_date[0].year - w.creation_date[0].year
            w.expiration_date = w.expiration_date[0]
            w.creation_date = w.creation_date[0]
        except:
            d = 0
    # no expiration date or no creation date
    if not w.expiration_date or not w.creation_date:
        return True
    # difference <= 1 year or difference more than 200 years
    if d <= 1 or d > 200:
        return True
    # current date exceeds expiration date
    if datetime.now() > w.expiration_date:
        return True
    try:
        org = w.org
        reg = w.registrar
        email = w.emails
        status = w.status
        if org is None or reg is None or email is None or status is None:
            return True
    except:
        return True
    return False

In [9]:
# Checking non-phishing site
url = "https://www.google.com/"
check_whois(url)

False

In [10]:
# Checking phishing site
urls = ["https://cbsujavrh.ts.r.appspot.com/login.microsoftonline.com/common/oauth2", "https://www.netflix.verfiy-your-information.com/"]
for url in urls:
  print(check_whois(url))

True
True


## Google Search

In [11]:
def check_google(url):
    index = -1
    rslt = search(url, num_results=0)
    if len(rslt) != 0:
        index = rslt[0].find('://')
        if index != -1:
            rslt = rslt[0][index + 3:]
        index = rslt.find('/')
        if index != -1:
            rslt = rslt[:index]
        index = url.find('://')
        if index != -1:
            url = url[index + 3:]  
        index = url.find('/')
        if index != -1:
            url = url[:index]
    if rslt == url:
        return False
    return True

In [13]:
# Checking non-phishing site
url = "www.google.com"
check_google(url)

False


In [14]:
# Checking phishing site
urls = ["https://cbsujavrh.ts.r.appspot.com/login.microsoftonline.com/common/oauth2", "http://continouscrusader.com/www.amazon.com/preferredmethods/payment/Amazon", "http://tiny.cc/m5md0y?platform=hootsuite"]
for url in urls:
    print(check_google(url))

True
True
True


## ML Model

In [15]:
# Filter the uneccesary warnings
import warnings
warnings.filterwarnings("ignore")

# Import pandas and numpy
import pandas as pd
import numpy as np

In [16]:
data = pd.read_csv('dataset.csv')
data.head(10)

,index,having_IPhaving_IP_Address,URLURL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,2,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,3,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,4,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,5,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
5,6,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1
6,7,1,0,-1,1,1,-1,-1,-1,1,...,1,1,1,-1,-1,-1,1,0,-1,-1
7,8,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,0,-1,1,0,1,-1
8,9,1,0,-1,1,1,-1,1,1,-1,...,1,1,1,-1,1,1,1,0,1,1
9,10,1,1,-1,1,1,-1,-1,1,-1,...,1,1,1,-1,0,-1,1,0,1,-1
